# Claude를 사용하여 SQL 쿼리 생성하기


이 노트북에서는 자연어 질문을 바탕으로 Claude를 활용하여 SQL 쿼리를 생성하는 방법을 탐구합니다. 테스트 데이터베이스를 구축하고 해당 스키마를 Claude에 제공한 후, Claude가 자연어를 이해하고 이를 SQL 쿼리로 변환하는 과정을 시연합니다.

## 환경 설정

먼저, 필요한 라이브러리를 설치하고 API 키를 사용하여 Anthropic 클라이언트를 설정합니다.

In [ ]:
# 필요한 라이브러리를 설치합니다.
%pip install anthropic

In [2]:
# 필요한 라이브러리를 가져옵니다.
from anthropic import Anthropic
import sqlite3

# Anthropic API 클라이언트를 설정합니다.
client = Anthropic()
MODEL_NAME = "claude-3-opus-20240229"

## 테스트 데이터베이스 생성

SQLite를 사용하여 테스트 데이터베이스를 생성하고 샘플 데이터로 채워보겠습니다.

In [3]:
# 테스트 데이터베이스에 연결합니다 (데이터베이스가 존재하지 않으면 새로 생성됩니다).
conn = sqlite3.connect("test_db.db")
cursor = conn.cursor()

# 샘플 테이블을 생성합니다.
cursor.execute("""
    CREATE TABLE IF NOT EXISTS employees (
        id INTEGER PRIMARY KEY,
        name TEXT,
        department TEXT,
        salary INTEGER
    )
""")

# 샘플 데이터를 삽입합니다.
sample_data = [
    (1, "John Doe", "Sales", 50000),
    (2, "Jane Smith", "Engineering", 75000),
    (3, "Mike Johnson", "Sales", 60000),
    (4, "Emily Brown", "Engineering", 80000),
    (5, "David Lee", "Marketing", 55000)
]
cursor.executemany("INSERT INTO employees VALUES (?, ?, ?, ?)", sample_data)
conn.commit()

## Claude를 활용한 SQL 쿼리 생성

이제 자연어 질문을 Claude에 전송하고, 생성된 SQL 쿼리를 반환받는 함수를 정의하겠습니다.

In [8]:
# Claude에 질문을 보내고 응답을 받는 함수를 정의합니다.
def ask_claude(query, schema):
    prompt = f"""다음은 제공된 데이터베이스 스키마입니다:

{schema}

이 스키마를 바탕으로, 다음 질문에 답변할 수 있는 SQL 쿼리를 작성해주십시오. SQL 쿼리 외에는 다른 어떤 내용도 출력하지 마십시오.

질문: {query}
"""

    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2048,
        messages=[{
            "role": 'user', "content":  prompt
        }]
    )
    return response.content[0].text

데이터베이스 스키마를 조회하여 문자열 형식으로 변환합니다.

In [5]:
# 데이터베이스 스키마를 가져옵니다.
schema = cursor.execute("PRAGMA table_info(employees)").fetchall()
schema_str = "CREATE TABLE EMPLOYEES (\n" + "\n".join([f"{col[1]} {col[2]}" for col in schema]) + "\n)"
print(schema_str)

CREATE TABLE EMPLOYEES (
id INTEGER
name TEXT
department TEXT
salary INTEGER
)


이제, 예시 자연어 질문을 Claude에 전송해 보겠습니다.

In [9]:
# 자연어 질문 예시
question = "Engineering 부서에 속한 직원들의 이름과 급여는 어떻게 되나요?"
# 질문을 Claude에 전송하고 SQL 쿼리를 받습니다.
sql_query = ask_claude(question, schema_str)
print(sql_query)

SELECT name, salary
FROM EMPLOYEES
WHERE department = 'Engineering';


## 생성된 SQL 쿼리 실행하기

마지막으로, 생성된 SQL 쿼리를 테스트 데이터베이스에 대해 실행하고 그 결과를 출력합니다.

In [10]:
# SQL 쿼리를 실행하고 그 결과를 출력합니다.
results = cursor.execute(sql_query).fetchall()

for row in results:
    print(row)

('Jane Smith', 75000)
('Emily Brown', 80000)


모든 작업이 완료되면 데이터베이스 연결을 닫는 것을 잊지 마십시오.

In [11]:
# 데이터베이스 연결을 닫습니다.
conn.close()